In [98]:
import json
from urllib.request import urlopen
import requests

import pandas as pd
import numpy as np

import re

import itertools


def contains(sublist, item):
    for l in sublist:
        if re.search(r".*{}.*".format(item.lower()), l.lower()):
            return True
    return False


In [146]:
api = "https://openlibrary.org/search.json?subject=Ancient+History&page="

book_data = []

for page in range(1,300):
    print(page)
    res = urlopen(api + str(page))
    book_data+=json.load(res)['docs']
    
    

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [151]:

book_data[500]



{'key': '/works/OL21254571W',
 'type': 'work',
 'seed': ['/books/OL29280423M',
  '/books/OL29506906M',
  '/books/OL29278240M',
  '/books/OL29390973M',
  '/books/OL28775927M',
  '/books/OL33784564M',
  '/works/OL21254571W',
  '/subjects/philosophy_ancient',
  '/subjects/science_philosophy',
  '/subjects/blood',
  '/subjects/ancient_science',
  '/subjects/ancient_philosophy',
  '/subjects/science',
  '/subjects/philosophy',
  '/subjects/history',
  '/subjects/science',
  '/authors/OL7696893A'],
 'title': 'Origins of Ancient Greek Science',
 'title_suggest': 'Origins of Ancient Greek Science',
 'title_sort': 'Origins of Ancient Greek Science',
 'edition_count': 6,
 'edition_key': ['OL29280423M',
  'OL29506906M',
  'OL29278240M',
  'OL29390973M',
  'OL28775927M',
  'OL33784564M'],
 'publish_date': ['2015', '2019'],
 'publish_year': [2015, 2019],
 'first_publish_year': 2015,
 'number_of_pages_median': 170,
 'lccn': ['2015937666'],
 'oclc': ['907773435', '896601522'],
 'lcc': ['Q--0127.00000

In [152]:
df = pd.DataFrame(book_data)[[
    'title', 'author_name',
    'first_publish_year', 'publish_year',
    'place', 'place_key',
    'subject', 'subject_key', 
    'person', 'person_key', 
    'language',
    'isbn',
    'key']]

book_url = "https://openlibrary.org"
df['url']=book_url + df['key']


df.rename(columns={'publish_year': 'last_publish_year'}, inplace=True)

df['author_name'] = df['author_name'].fillna('N/A').apply(lambda x: x[0])



df['last_publish_year'] = df['last_publish_year'].fillna('N/A').apply(lambda x: x[-1])

df['last_publish_year'] = pd.to_numeric(df['last_publish_year'], errors='coerce')



df = df.query('last_publish_year > 2000')

df['first_publish_year'] =  df['first_publish_year'].astype('int')
df['last_publish_year']  =  df['last_publish_year'].astype('int')

df['isbn']=df['isbn'].fillna('N/A').apply(lambda x: x[-1])
df = df[df['language'].fillna('').apply(contains, item='eng')]

df.to_json('data/downloaded.json')

#df.head()

! cp data/downloaded.json data/data.json
! cp data/downloaded.json data/data.json.bak

In [218]:
df = pd.read_json('data/downloaded.json')


In [153]:
import geopy
from geopy.geocoders import Nominatim

# Create a geocoder object
geocoder = Nominatim(user_agent="http")

# Get the country name from the coordinates
latitude = 52.82932091031373 
longitude = -2.1752929687500004
location = geocoder.reverse(f"{latitude},{longitude}", language='en')
country = location.raw.get('address').get('country')
state = location.raw.get('address').get('state')


print(country,"-", state)  # Output: United States


United Kingdom - England


In [167]:
filtered_df = pd.concat([
    df[df['place_key'].fillna('').apply(contains, item=country)], 
    df[df['place_key'].fillna('').apply(contains, item=state)]
    ]).drop_duplicates(subset=['title'])

filtered_df = filtered_df[['title', 'author_name', 'last_publish_year', 'url']]

def make_clickable(link):
    text = link.split('===')[1]
    url =  link.split('===')[0]
    return f'<a target="_blank" href="{url}">{text}</a>'



(filtered_df['url']+ "===" + filtered_df['title']).apply(make_clickable)

98       <a target="_blank" href="https://openlibrary.o...
494      <a target="_blank" href="https://openlibrary.o...
1117     <a target="_blank" href="https://openlibrary.o...
2074     <a target="_blank" href="https://openlibrary.o...
2392     <a target="_blank" href="https://openlibrary.o...
3067     <a target="_blank" href="https://openlibrary.o...
3071     <a target="_blank" href="https://openlibrary.o...
3729     <a target="_blank" href="https://openlibrary.o...
5723     <a target="_blank" href="https://openlibrary.o...
8064     <a target="_blank" href="https://openlibrary.o...
9251     <a target="_blank" href="https://openlibrary.o...
10188    <a target="_blank" href="https://openlibrary.o...
10444    <a target="_blank" href="https://openlibrary.o...
10514    <a target="_blank" href="https://openlibrary.o...
10701    <a target="_blank" href="https://openlibrary.o...
10917    <a target="_blank" href="https://openlibrary.o...
12484    <a target="_blank" href="https://openlibrary.o.

In [205]:
list('hello')

['h', 'e', 'l', 'l', 'o']

In [219]:

def contains(sublist, item):
    if item is None:
        return False
    for l in sublist:
        for i in item:
            if re.search(r".*{}.*".format(i.lower()), l.lower()):
                return True
    return False

In [230]:
all_names={'iran': ['iran','persia','persepolis', 'pasargad', 'elam', 'media']}

country='Iran'.lower()
if country in all_names.keys():
    filtered_df = df[df['place_key'].fillna('').apply(contains, item=all_names[country])]
else:
    filtered_df = df[df['place_key'].fillna('').apply(contains, item=[country])]

In [234]:
filtered_df.reset_index()

,index,title,author_name,first_publish_year,last_publish_year,place,place_key,subject,subject_key,person,person_key,language,isbn,key,url
0,593,Persians,Lloyd Llewellyn-Jones,2022,2022,"[Babylon, Egypt, Pasargadae, Persepolis]","[babylon, egypt, pasargadae, persepolis]","[Persia, ancient history, Asian / Middle Easte...","[ancient_history, asian__middle_eastern_histor...","[Alexander, Artaxerxes, Cambyses, Cyrus, Dariu...","[alexander, artaxerxes, cambyses, cyrus, dariu...",[eng],9781541604230,/works/OL25338087W,https://openlibrary.org/works/OL25338087W
1,3185,Iran,Giovanni Curatola,2003,2007,[Iran],[iran],"[Art, Iranian, Iranian Art, Architecture, Hist...","[architecture, architecture_(general), archite...",None,None,"[ita, eng]",8816605693,/works/OL12438935W,https://openlibrary.org/works/OL12438935W
2,5049,Excavating an empire,Touraj Daryaee,2014,2014,[Iran],[iran],"[History, Ancient History]","[ancient_history, history]",[Achaemenid dynasty (559-330 B.C)],[achaemenid_dynasty_(559-330_b.c)],[eng],9781568592985,/works/OL23318927W,https://openlibrary.org/works/OL23318927W
3,5207,The Persian empire,"Allen, Lindsay.",2005,2005,[Iran],[iran],"[Civilization, History, History, ancient, Perz...","[civilization, histoire, history, history_anci...",None,None,[eng],9780226014470,/works/OL5844839W,https://openlibrary.org/works/OL5844839W
4,6528,Persian art,Vladimir Loukonine,1998,2012,[Iran],[iran],"[Ancient Art, Iranian Art, History, Catalogs, ...","[ancient_art, art, catalogs, history, iranian_...",None,None,[eng],9781859951675,/works/OL20325135W,https://openlibrary.org/works/OL20325135W
5,6923,Arsacids and Sasanians,M. Rahim Shayegan,2011,2011,"[Rome, Iran]","[iran, rome]","[History, Sassanids, HISTORY / Ancient / Gener...","[history, history__ancient__general, iran_hist...",[Arsacid dynasty (247 B.C.-224 A.D)],[arsacid_dynasty_(247_b.c.-224_a.d)],[eng],9780521766418,/works/OL16414417W,https://openlibrary.org/works/OL16414417W
6,7118,Arsacids and Sasanians,M. Rahim Shayegan,2011,2011,"[Rome, Iran]","[iran, rome]","[History, Sassanids, HISTORY / Ancient / General]","[history, history__ancient__general, sassanids]",[Arsacid dynasty (247 B.C.-224 A.D)],[arsacid_dynasty_(247_b.c.-224_a.d)],[eng],9780521766418,/works/OL16414416W,https://openlibrary.org/works/OL16414416W
7,9127,A Walk Through the Iranian Heavens,Antonio Panaino,2019,2019,[Iran],[iran],"[Astronomy, History, Cosmography, Ancient Civi...","[ancient_civilization, astronomy, cosmography,...",None,None,[eng],9004460691,/works/OL32300508W,https://openlibrary.org/works/OL32300508W
8,9520,Splendors of the ancient Persia,Henri Stierlin,2006,2006,[Iran],[iran],"[Achaemenid Art, Parthian Art, Civilization, A...","[achaemenid_art, ancient_art, antiquities, art...",None,None,[eng],8854401463,/works/OL19017218W,https://openlibrary.org/works/OL19017218W
9,9682,Ancient Greece and ancient Iran,"Greece) International Conference ""Ancient Gree...",2008,2008,"[Iran, Greece]","[greece, iran]","[Ancient History, Relations, History, Congress...","[ancient_history, antiquities, congresses, his...",None,None,[eng],9789609309554,/works/OL22400473W,https://openlibrary.org/works/OL22400473W


In [215]:
#df[df['place_key'].fillna('').apply(contains, item=country)].head(2)

df[df['place_key'].fillna('').apply(contains, item=all_names[country])]

,title,author_name,first_publish_year,last_publish_year,place,place_key,subject,subject_key,person,person_key,language,isbn,key,url
593,Persians,Lloyd Llewellyn-Jones,2022,2022,"[Babylon, Egypt, Pasargadae, Persepolis]","[babylon, egypt, pasargadae, persepolis]","[Persia, ancient history, Asian / Middle Easte...","[ancient_history, asian__middle_eastern_histor...","[Alexander, Artaxerxes, Cambyses, Cyrus, Dariu...","[alexander, artaxerxes, cambyses, cyrus, dariu...",[eng],9781541604230,/works/OL25338087W,https://openlibrary.org/works/OL25338087W
11443,Elamite and Achaemenid settlement on the Deh L...,H. T. Wright,2010,2010,"[Deh Luran Plain (Iran), Elam, Iran, Deh Luran...","[deh_luran_plain, deh_luran_plain_(iran), elam...","[Antiquities, Local History, Ancient Cities an...","[ancient_cities_and_towns, ancient_pottery, an...",[Achaemenid dynasty (559-330 B.C)],[achaemenid_dynasty_(559-330_b.c)],[eng],9780915703722,/works/OL23262805W,https://openlibrary.org/works/OL23262805W


In [132]:


pd.concat([df[df['place_key'].fillna('').apply(contains, item=country)], df[df['place_key'].fillna('').apply(contains, item=state)]]).drop_duplicates(subset=['title'])

,title,author_name,first_publish_year,last_publish_year,place,place_key,subject,subject_key,person,person_key,language,isbn
97,The antiquities of Athens,"Stuart, James",1762,2008,"[Athens, Athens (Greece), England, Greece]","[athens, athens_(greece), england, greece]","[Antiquities, Architecture, Buildings, structu...","[ancient_greece, antiquities, architecture, ar...",None,None,"[eng, fre]",9781376149005
493,"Commentaries on the history, constitution, and...",George Norton,1829,2007,[London (England)],[london_(england)],"[Politics and government, Charters, grants, pr...","[ancient, charter, charters_grants_privileges,...",None,None,[eng],9781432680534
1116,Stonehenge,Francis Pryor,2016,2018,"[Stonehenge (England), Wiltshire, England, Wil...","[england, stonehenge_(england), wiltshire, wil...","[Engineering, Ancient History, Megalithic monu...","[ancient_history, antiquities, engineering, me...",None,None,[eng],9781681777030
2069,Ovid and the cultural politics of translation ...,Liz Oakley-Brown,2006,2006,[England],[england],"[Adaptations, Appreciation, English literature...","[adaptations, ancient__classical, appreciation...",[Ovid (43 B.C.-17 or 18 A.D)],[ovid_(43_b.c.-17_or_18_a.d)],[eng],9780754651550
2388,Uriel's machine,"Knight, Christopher",1999,2001,"[England, Europa, Europe, Great Britain, Ingla...","[england, europa, europe, great_britain, ingla...","[Ancient Astronomy, Ancient Civilization, Anti...","[ancient_-_general, ancient_astronomy, ancient...",None,None,"[spa, eng]",1446410447
3066,"Wroxeter, the Cornovii, and the urban process",V. L. Gaffney,2006,2006,"[Wroxeter, Wroxeter (England), Great Britain, ...","[england, great_britain, shropshire, wroxeter,...","[Urbanization, Romans, Excavations (Archaeolog...","[ancient_cities_and_towns, cities_and_towns_an...",None,None,[eng],9781887829687
3070,The city,Kathryn Hinds,2004,2006,"[Islamic countries, England, Europe, Great Bri...","[egypt, england, europe, great_britain, islami...","[Juvenile literature, Social life and customs,...","[ancient_cities_and_towns, cities_and_towns, c...",None,None,[eng],0761416781
3744,"Archaeology, economy, and society",David Alban Hinton,2002,2022,[England],[england],"[Ancient, Antiquities, General, Economic condi...","[ancient, antiquities, economic_conditions, en...",None,None,[eng],0203264304
5716,The Dover Bronze Age boat in context,Béat Arnold,2004,2004,"[Dover (England), Dover, England, Northern Eur...","[dover, dover_(england), england, northern_eur...","[Prehistoric Boats, Prehistoric Navigation, Br...","[ancient_boats, ancient_ships, antiquities, an...",None,None,"[und, eng]",9781842171394
8053,Rider in the Dark,Victoria Holmes,2004,2006,"[Dorset (England), Great Britain]","[dorset_(england), great_britain]","[Fiction, History, Horses, Smugglers, Juvenile...","[children's_fiction, england_fiction, fiction,...",None,None,[eng],9780060520274


In [106]:
countries = df['place_key'].fillna('').copy()



In [96]:

all_countries = []
for country in countries.values.flatten():
    all_countries.append(country)

all_countries

[['greece'],
 '',
 ['greece', 'rome', 'rome_(empire)'],
 '',
 ['russia'],
 ['ilium', 'troy', 'troy_(extinct_city)', 'turkey'],
 ['russia'],
 '',
 ['athens_(greece)'],
 '']

In [108]:
flat_list = list(itertools.chain.from_iterable(countries.tolist()))
unique_list = list(set(flat_list))
print(unique_list)

['britain', 'stonehenge_world_heritage_site_(england)', 'wiltshire', 'ilium', 'western_mediterranean', 'elam', 'north_africa', 'aztec', 'california', 'teotihuacán_site_(san_juan_teotihuacán_mexico)', 'romerska_riket', "hadrian's_wall_(england)", 'dakhla_oasis_(egypt)', 'the_nile', 'industal', 'ljubljanica_river_region_(slovenia)', 'andalusia', 'kings_lynn', 'gr©·ce', 'thailand', 'region_naples_bay_of', 'athens_(greece)', 'zagros_mountains_(iran)', 'vaihingen_an_der_enz_(germany)', 'mytilene', 'kemalpaşa_(i̇zmir_i̇li_turkey)', 'grèce', 'indien', 'massachusetts', 'bagdad', 'greeks', 'naples_region_(italy)', 'great_britain', 'south_africa', 'kahun_(extinct_city)', 'north_macedonia', 'georgia', 'shāriqah', 'laodicea_ad_lycum_(extinct_city)', 'spain', 'asea_valley_(greece)', 'new_york', 'bering_strait_region', 'babylon', 'ercolano', 'grèce_antique', 'bithynia', 'iraq', 'angleterre', 'central_europe', 'euphrates_valley', 'vesuvius_(italy)', 'asia_minor', 'tombouctou_(mali)', 'denmark', 'balt

In [68]:
df

,col1
0,"[a, b, c]"
1,"[d, e, f]"
2,"[g, h, i]"


In [116]:
import geopy
from geopy.geocoders import Nominatim

# Create a geocoder object
geocoder = Nominatim(user_agent="http")

# Get the country name from the coordinates
latitude = 57.27904276497778
longitude = -4.658203125000001
location = geocoder.reverse(f"{latitude},{longitude}", language='en')
country = location.raw.get('address').get('country')
state = location.raw.get('address').get('state')


print(country,"-", state)  # Output: United States


United Kingdom - Scotland


In [109]:
import pandas as pd
import geopy
from geopy.geocoders import Nominatim

# Create a pandas dataframe with coordinates
data = {'latitude': [40.712784, 37.774929, 41.008618, 51.507281, 38.4106],
        'longitude': [-74.005942, -122.419416, -87.623163, 10.443060, 33.0469]}
df = pd.DataFrame(data)

# Create a geocoder object
geocoder = Nominatim(user_agent="http")

# Get the country name for each coordinate pair
df['country'] = [geocoder.reverse(f"{lat},{lon}", language='en').raw.get('address').get('country') for lat, lon in zip(df['latitude'], df['longitude'])]

df


,latitude,longitude,country
0,40.712784,-74.005942,United States
1,37.774929,-122.419416,United States
2,41.008618,-87.623163,United States
3,51.507281,10.443060,Germany
4,38.410600,33.046900,Turkey


In [111]:
geocoder.reverse("57.27904276497778,-4.658203125000001", language='en').raw.get('address')

{'county': 'Highland',
 'ISO3166-2-lvl6': 'GB-HLD',
 'state': 'Scotland',
 'ISO3166-2-lvl4': 'GB-SCT',
 'country': 'United Kingdom',
 'country_code': 'gb'}

In [237]:


df_fav = pd.DataFrame({'title': ['a', 'b', 'c', 'd', 'e'], 'favorite': [True, True, False, True, True]})
edited_df = pd.DataFrame({'title': ['a', 'b', 'c', 'f', 'g'], 'favorite': [True, False, True, False, False]})

def g(df_fav, edited_df):
    df_fav = df_fav.loc[~df_fav['title'].isin(edited_df[edited_df['favorite'] == False]['title'])]
    return df_fav

df_fav = g(df_fav.copy(), edited_df.copy())
print(df_fav)


  title  favorite
0     a      True
2     c     False
3     d      True
4     e      True
